# 連接富途牛牛API,模擬交易

In [ ]:
import math
import time
import pandas as pd
import decimal

from futu import *


#FUTU
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
trd_ctx = OpenFutureTradeContext(host='127.0.0.1', port=11111)
pwd_unlock = 'xxxx'
trd_ctx.unlock_trade(pwd_unlock)


p=0.4/100
class PriceReminderTest(PriceReminderHandlerBase):
    def on_recv_rsp(self, rsp_pb):
        ret_code, content = super(PriceReminderTest, self).on_recv_rsp(rsp_pb)
        if ret_code != RET_OK:
            print("PriceReminderTest: error, msg: %s" % content)
            return RET_ERROR, content
        print("PriceReminderTest ", content)  # PriceReminderTest 自己的處理邏輯
        print(content['note'])
        try:
         info = eval(content['note'])
         print(info)

        except:
            pass
        # content['set_value']是現價
        if 'x' in content['note']:
            print('1')
            pass

        elif content['note'] in 'go':
            print('2')
            info = [content['set_value'],0,0,0]
            content['note'] = str(info)
            print(content['note'])
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.DEL_ALL, key=None,
                                         reminder_type=PriceReminderType.PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=1, note='dele all')  # dele all
            '-------------------'
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.PRICE_UP,
                                         reminder_freq=PriceReminderFreq.ONCE, value=round(content['set_value'] * (1+p), 3),
                                         note='x')

            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=round(content['set_value'] * (1-p), 3),
                                         note='x')

            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.BID_PRICE_UP,
                                         reminder_freq=PriceReminderFreq.ONCE,
                                         value=round(content['set_value'] * (1 + p), 3),
                                         note=content['note'])

            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.ASK_PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE,
                                         value=round(content['set_value'] * (1 - p), 3),
                                         note=content['note'])


        elif info[1] == 0 and info[0] == content['set_value']:
            print('3')
            # 在BP價
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.DEL_ALL, key=None,
                                         reminder_type=PriceReminderType.PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=1, note='dele all')  # dele all
            '----------------------'
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.PRICE_UP,
                                         reminder_freq=PriceReminderFreq.ONCE, value=round(content['set_value'] * (1+p),3),
                                         note='x')
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=round(content['set_value'] * (1-p),3),
                                         note='x')
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.BID_PRICE_UP,
                                         reminder_freq=PriceReminderFreq.ONCE,
                                         value=round(content['set_value'] * (1 + p), 3),
                                         note=content['note'])
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.ASK_PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE,
                                         value=round(content['set_value'] * (1 - p), 3),
                                         note=content['note'])

        elif info[1] == 0 and content['set_value'] >= round(info[0]*(1+p),3):
            print('4')
            #'開多倉，BP*(1+p)'
            symbol_new = content['code']
            open_price_new = info[0]
            quantity_new = 2**info[3]
            print(symbol_new)
            print(open_price_new)
            print(quantity_new)
            # print(trd_ctx.place_order(price=10, qty=quantity_new , code=symbol_new, trd_side=TrdSide.BUY,
            #                           order_type=OrderType.MARKET))
            #'模擬開多倉'
            print('模擬開多倉'+str(content['code'])+',qty,'+str(quantity_new))
            #'修改note'
            info[1]= quantity_new
            info[3]+= 1
            note_2= str(info)
            #'設置提醒'
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.DEL_ALL, key=None,
                                         reminder_type=PriceReminderType.PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=1, note='dele all')  # dele all
            '---------------'
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.PRICE_UP,
                                         reminder_freq=PriceReminderFreq.ONCE, value=round(info[0]* (1+3*p), 3),
                                         note='x')
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=info[0],
                                         note='x')
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.BID_PRICE_UP,
                                         reminder_freq=PriceReminderFreq.ONCE, value=round(info[0] * (1 + 3 * p), 3),
                                         note=note_2)
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.ASK_PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=info[0],
                                         note=note_2)

        elif info[1] == 0 and content['set_value'] <= round(info[0]*(1-p),3):
            print('5')
            #'開空倉，BP*(1-p)'
            symbol_new = content['code']
            open_price_new = info[0]
            quantity_new = (-1)* 2 ** info[3]
            # print(trd_ctx.place_order(price=10, qty=quantity_new, code=symbol_new, trd_side=TrdSide.SELL,
            #                           order_type=OrderType.MARKET))
            #'模擬開空倉'
            print('模擬開空倉'+str(content['code'])+',qty,'+str(quantity_new))
            #'修改note'
            info[1] = quantity_new
            info[3] += 1
            note_2 = str(info)
            #'設置提醒'
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.DEL_ALL, key=None,
                                         reminder_type=PriceReminderType.PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=1, note='dele all')  # dele all
            '------------------------'
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.PRICE_UP,
                                         reminder_freq=PriceReminderFreq.ONCE, value=info[0],
                                         note='x')
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=round(info[0]*(1-3*p),3),
                                         note='x')
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.BID_PRICE_UP,
                                         reminder_freq=PriceReminderFreq.ONCE, value=info[0],
                                         note=note_2)
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.ASK_PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=round(info[0] * (1 - 3 * p), 3),
                                         note=note_2)

        elif info[1] != 0 and info[0] != content['set_value']:
            print('6')
            #'模擬止盈'
            print('模擬止盈' +str(content['code'])+',qty,' + str(info[1]))
            #'設置提醒'
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.DEL_ALL, key=None,
                                         reminder_type=PriceReminderType.PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=1, note='dele all')  # dele all

        elif info[1] != 0 and info[0] == content['set_value']:
            print('7')
            #'止損'
            #'模擬止損'
            print('模擬止損'+str(content['code'])+',qty,'+str(info[1]))
            #'修改note'
            info[2] =abs(info[1])*(-1)
            info[1] = 0
            note_2 = str(info)
            #'設置提醒'
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.DEL_ALL, key=None,
                                         reminder_type=PriceReminderType.PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=1, note='dele all')  # dele all
            '-----------------'
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.PRICE_UP,
                                         reminder_freq=PriceReminderFreq.ONCE, value=round(info[0]* (1+p), 3),
                                         note='x')
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=round(info[0]* (1-p), 3),
                                         note='x')
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.BID_PRICE_UP,
                                         reminder_freq=PriceReminderFreq.ONCE, value=round(info[0] * (1 + p), 3),
                                         note=note_2)
            quote_ctx.set_price_reminder(code=content['code'], op=SetPriceReminderOp.ADD, key=None,
                                         reminder_type=PriceReminderType.ASK_PRICE_DOWN,
                                         reminder_freq=PriceReminderFreq.ONCE, value=round(info[0] * (1 - p), 3),
                                         note=note_2)
        else:
            print('error')

        return RET_OK, content
handler = PriceReminderTest()
quote_ctx.set_handler(handler)  # 设置到价提醒通知回调
# time.sleep(1)  # 设置脚本接收 FutuOpenD 的推送持续时间为15秒
# quote_ctx.close()   # 关闭当条连接，FutuOpenD 会在1分钟后自动取消相应股票相应类型的订阅


# bracket_order_buy(symbol_new,open_price_new,quantity_new)
# bracket_order_sell(symbol_new,open_price_new,quantity_new)
# app.disconnect()









